In [1]:
from qcg.generators import gen_supremacy
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.extensions.standard import HGate, SGate, SdgGate, XGate
from qiskit.tools.visualization import dag_drawer
from qiskit.quantum_info.states.measures import state_fidelity
import randomized_searcher as r_s
import pickle
import numpy as np
from qcg.generators import gen_supremacy

def measure_basis(l):
    if len(l)==1:
        return l[0]
    else:
        return np.kron(l[0], measure_basis(l[1:]))
    
initializations = (5,5,6)

complete_path_map = pickle.load( open( './data/cpm.p', 'rb' ) )
[print(x, complete_path_map[x]) for x in complete_path_map]
cluster_0_measurement = pickle.load( open( './data/cluster_0_measurement.p', 'rb' ) )
cluster_0_circ = pickle.load( open( './data/cluster_0_circ.p', 'rb' ) )
print(cluster_0_circ.qubits)

print('*'*100)

H = [[1,1],[1,-1]]/np.sqrt(2)
sDagger = [[1,0],[0,-1j]]
Id = [[1,0],[0,1]]

meas_0 = cluster_0_measurement[(1,)+initializations]
meas_1 = cluster_0_measurement[(5,)+initializations]

prod = measure_basis([Id,Id,Id,Id,Id,Id,np.matmul(H,sDagger)])

modified = np.matmul(prod, meas_0)
print('fidelity between modified vs simulation =',state_fidelity(modified, meas_1))

print('*'*100)
cluster_0_measurement_sv, cluster_0_cut_in_qubits = pickle.load( open( './data/cluster_0_measurement_sv.p', 'rb' ) )
meas_2 = cluster_0_measurement_sv[initializations]
sv_modified = np.matmul(prod, meas_2)
print('fidelity between sv_modified vs simulation', state_fidelity(sv_modified, meas_1))

ModuleNotFoundError: No module named 'simulator'